<a href="https://colab.research.google.com/github/hypro2/hands-on-langchain-langgraph/blob/main/pdf_to_md.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG 시스템을 위한 PDF → Markdown 변환

## 소개

PDF 문서를 Markdown으로 변환하는 과정은 효과적인 RAG(Retrieval-Augmented Generation) 시스템을 구축하는 데 있어 종종 **가장 핵심적인 단계**입니다. Markdown은 최적의 균형점을 제공합니다. 원본 문서의 구조적 계층(헤더, 목록, 표, 코드, 수식)을 유지하면서도 가볍고, 추가 전처리 없이 LLM이 바로 소비할 수 있기 때문입니다.

**왜 일반 텍스트나 JSON이 아니라 Markdown인가?**

일반 텍스트 추출은 빠르고 JSON은 구조화된 데이터를 제공하지만, **RAG 시스템에서는 Markdown이 더 우수**합니다. 그 이유는 다음과 같습니다:

- **문서 구조 보존**: 헤더, 서브헤더, 목록, 서식 계층이 그대로 유지되어 LLM이 섹션 간의 논리적 흐름과 관계를 이해하는 데 도움이 됩니다
- **의미적 맥락 유지**: 굵게, 기울임, 코드 블록 등 다양한 서식 신호가 문맥을 제공하여 이해도를 높입니다
- **복잡한 요소의 자연스러운 처리**: 표, 코드 블록, 수학 수식, 인용문 등이 표준화되고 읽기 쉬운 형식으로 표현됩니다
- **사람과 기계 모두에게 가독성 제공**: JSON의 경직된 키-값 구조나 일반 텍스트의 계층 부재와 달리, Markdown은 사람과 LLM 모두에게 읽기 좋은 균형을 제공합니다
- **청킹(chunking)에 최적화**: 명확한 구조적 경계(헤더, 문단)를 통해 검색을 위한 지능적인 문서 청킹이 용이합니다

일반 텍스트는 모든 서식과 구조를 잃어버리기 때문에 검색 시스템이 제목, 본문, 메타데이터를 구분하기 어렵습니다. JSON은 구조를 담을 수 있지만, 종종 커스텀 스키마가 필요하고 텍스트 중심 문서에는 장황해집니다. Markdown은 이 간극을 자연스럽게 메워줍니다.

**이 변환이 왜 그렇게 중요한가요?**

RAG 시스템의 품질은 근본적으로 추출된 데이터의 품질에 의해 제한됩니다. 서식이 깨진 텍스트, 누락된 표, 망가진 수식, 맥락이 사라진 데이터와 같은 저품질 또는 “더러운” 데이터는 부정확한 검색 결과와 환각(hallucination) 응답으로 직결됩니다. 어떤 추출 파이프라인을 구현하기 전에 반드시 다음 질문을 던져야 합니다:

- **PDF에는 어떤 유형의 콘텐츠가 포함되어 있는가?** 일반 텍스트만 있는가? 이미지가 있는가? 표가 있는가? 수학 수식이 있는가?
- **레이아웃은 얼마나 복잡한가?** 단일 컬럼인가? 다중 컬럼인가? 사이드바가 섞인 혼합 레이아웃인가?
- **시각적 요소가 있는가?** 다이어그램, 차트, 사진 등 의미를 담고 있는 요소가 있는가?
- **문서가 스캔본인가?** 스캔된 PDF는 레이아웃 복잡도와 무관하게 OCR 기능이 필요합니다.

이 질문들을 바탕으로 PDF를 세 가지 복잡도 등급으로 분류할 수 있습니다:

---

## PDF 복잡도 분류

**🟢 단순 PDF (카테고리 1)**
- 표준 레이아웃의 텍스트 전용 문서
- 텍스트 선택이 가능한 디지털 PDF
- 예시: 보고서, 기사, 일반 서적, 문서 자료
- **참고:** 스캔본인 경우 카테고리 2로 이동 (OCR 필요)

**🟡 중간 복잡도 PDF (카테고리 2)**
- 표와 기본 서식이 포함된 문서
- 스캔된 문서(레이아웃이 단순하더라도)
- 간헐적인 이미지가 포함된 PDF
- 다중 컬럼 레이아웃
- 예시: 학술 논문, 비즈니스 보고서, 스캔된 서적

**🔴 고복잡도 PDF (카테고리 3)**
- 시각적 요소가 핵심 정보를 담고 있는 이미지 중심 문서
- 복잡한 차트, 다이어그램, 인포그래픽
- 공간적 관계를 가지는 혼합 콘텐츠 유형
- 예시: 다이어그램이 포함된 과학 논문, 의료 보고서, 기술 매뉴얼, 프레젠테이션

---

## 카테고리 1: 단순 PDF – 빠른 텍스트 추출

**이 접근 방식을 사용하는 경우:** PDF가 디지털 형식(스캔 아님)이며, 단순한 서식의 텍스트 위주이고, 중요한 시각적 요소가 없는 경우.

**권장 도구:**
- **PyMuPDF4LLM** – https://github.com/pymupdf/PyMuPDF4LLM
- **문서** – https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/index.html

### 예시 구현: PyMuPDF4LLM

In [1]:
!pip install -qU pymupdf4llm pymupdf PyMuPDF docling google-genai pathlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.0/304.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.6/238.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 47.1 MB/s eta 0:00:00


In [19]:
pdf_path = pathlib.Path('./pdfs')
pdf_path.mkdir(parents=True, exist_ok=True)

In [20]:
import pymupdf4llm
import pathlib

def convert_simple_pdfs_pymupdf4llm(pdf_folder: str, output_folder: str):
    """
    PyMuPDF4LLM을 사용하여 텍스트 기반 PDF 파일을 마크다운 형식으로 변환합니다.

    이 함수는 지정된 폴더 내의 모든 PDF 파일을 검색하여 각 파일의 구조와 내용을
    LLM 친화적인 마크다운 포맷으로 추출하고 저장하는 역할을 수행합니다.

    Args:
        pdf_folder (str): PDF 파일들이 위치한 소스 디렉토리 경로.
        output_folder (str): 변환된 마크다운 파일이 저장될 목적지 디렉토리 경로.

    Returns:
        None
    """
    # pathlib.Path 객체를 생성하여 경로 조작의 유연성 및 OS 호환성 확보
    pdf_path = pathlib.Path(pdf_folder)
    output_path = pathlib.Path(output_folder)

    # 출력 디렉토리가 존재하지 않을 경우 생성 (상위 디렉토리 포함)
    output_path.mkdir(parents=True, exist_ok=True)

    # 폴더 내의 모든 .pdf 확장자 파일을 반복적으로 탐색
    for pdf_file in pdf_path.glob("*.pdf"):
        try:
            # Step 1: PyMuPDF4LLM을 호출하여 PDF 콘텐츠를 마크다운 텍스트로 추출
            # str(pdf_file)을 통해 Path 객체를 문자열 경로로 전달
            md_text = pymupdf4llm.to_markdown(str(pdf_file))

            # Step 2: 출력 파일 경로 설정 (원본 파일명에서 확장자만 .md로 변경)
            output_file = output_path / f"{pdf_file.stem}.md"

            # Step 3: UTF-8 인코딩을 명시하여 추출된 마크다운 텍스트를 파일로 저장
            output_file.write_text(md_text, encoding='utf-8')

            print(f"✓ 변환 완료: {pdf_file.name}")

        except Exception as e:
            # 개별 파일 처리 중 발생하는 예외를 캡처하여 전체 루프의 중단을 방지
            print(f"✗ 오류 발생 ({pdf_file.name}): {e}")

    print(f"\n변환 작업이 완료되었습니다! 결과물 위치: '{output_folder}'")

# 실행 예시 (Example usage)
# './simple_pdfs' 폴더의 PDF들을 './md_output' 폴더로 변환
convert_simple_pdfs_pymupdf4llm("./pdfs", "./md_output_simple")

✓ 변환 완료: T_cn202310830097bp.pdf

변환 작업이 완료되었습니다! 결과물 위치: './md_output_simple'


---

## 카테고리 2: 중간 복잡도의 PDF - OCR + 구조 인식

**이 접근 방식을 사용해야 할 때:** 스캔된 문서, 구조 보존이 필요한 표, 또는 다단 레이아웃이 있는 경우.

**권장 도구:**
- **Docling** - https://github.com/DS4SD/docling  
- **문서** - https://docling-project.github.io/docling/

### 예시 구현: Docling

In [21]:
from pathlib import Path
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions

def convert_medium_pdfs_docling(pdf_folder: str, output_folder: str):
    """
    Docling 라이브러리를 사용하여 OCR 및 표 추출 기능이 포함된 정밀 PDF 변환을 수행합니다.

    단순 텍스트뿐만 아니라 스캔된 이미지 내의 텍스트(OCR)와 복잡한 표 구조를
    보존하며 마크다운 형식으로 변환하는 역할을 합니다.

    Args:
        pdf_folder (str): 입력 PDF 파일들이 저장된 디렉토리 경로.
        output_folder (str): 결과물 마크다운 파일이 저장될 디렉토리 경로.
    """
    # Step 1: PDF 분석을 위한 세부 파이프라인 옵션 설정
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_table_structure = True  # 표(Table) 구조 분석 및 추출 활성화
    pipeline_options.do_ocr = True              # 스캔된 문서 대응을 위한 OCR 활성화
    pipeline_options.images_scale = 2.0         # 이미지 추출 품질 향상을 위한 스케일링 설정
    pipeline_options.generate_picture_images = True # 문서 내 삽입된 그림(Picture) 이미지 생성 활성화

    # Step 2: 설정된 옵션을 바탕으로 문서 변환기(Converter) 인스턴스 생성
    converter = DocumentConverter(
        format_options={
            # PDF 입력 포맷에 대해 정의된 파이프라인 옵션 적용
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    # 출력 경로 객체 생성 및 물리적 디렉토리 존재 보장
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)

    # 지정된 폴더에서 모든 PDF 파일 리스트 확보
    pdf_files = list(Path(pdf_folder).glob("*.pdf"))

    for pdf_file in pdf_files:
        try:
            # Step 3: 문서를 변환하고 내부 객체 모델로 로드
            # convert() 함수는 레이아웃 분석 및 요소 분류를 수행함
            result = converter.convert(str(pdf_file))

            # Step 4: 분석된 문서 모델을 마크다운(Markdown) 문자열로 변환
            markdown_content = result.document.export_to_markdown()

            # 파일명은 유지하되 확장자만 .md로 변경하여 저장
            output_file = output_path / f"{pdf_file.stem}.md"
            output_file.write_text(markdown_content, encoding='utf-8')

            print(f"✓ 변환 성공: {pdf_file.name}")

        except Exception as e:
            # 변환 중 발생하는 예외 상황(파일 손상, 메모리 부족 등) 로깅
            print(f"✗ 처리 중 오류 발생 {pdf_file.name}: {e}")

    print(f"\n모든 변환 프로세스가 완료되었습니다! 출력 폴더: '{output_folder}'")

# 실행 예시
if __name__ == "__main__":
    convert_medium_pdfs_docling("./pdfs", "./md_output_middle")

[INFO] 2026-02-05 02:03:01,057 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-05 02:03:01,063 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-05 02:03:01,066 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.6.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-05 02:03:01,888 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2026-02-05 02:03:02,491 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-05 02:03:02,493 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-05 02:03:02,921 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-05 02:03:02,922 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-05 02:03:02,924 [RapidOCR] download_file.py:68: Initiat

✓ 변환 성공: T_cn202310830097bp.pdf

모든 변환 프로세스가 완료되었습니다! 출력 폴더: './md_output_middle'


---

## 카테고리 3: 복잡한 PDF - 비전-언어 모델(VLM)

**VLM을 사용해야 할 때:** PDF에 차트, 다이어그램, 복잡한 레이아웃, 이미지 중심 콘텐츠 등 중요한 시각적 정보가 포함되어 있으며, 이러한 시각 요소를 정확하게 설명하고 맥락화해야 할 경우.

### VLM 접근 방식

VLM 접근 방식은 각 PDF 페이지를 고해상도 이미지로 변환한 뒤, 콘텐츠를 구조화된 Markdown으로 추출·변환하도록 지시와 함께 비전-언어 모델에 전달하는 방식입니다. 이 방법은 모델의 시각적 이해 능력을 활용하여 다음을 수행합니다.

1. **어떤 레이아웃이나 방향에서도 텍스트 인식**
2. **차트, 다이어그램, 이미지와 같은 시각 요소 해석**
3. **문서 요소 간의 공간적 관계 이해**
4. **적절한 Markdown 포맷을 통한 문서 구조 보존**
5. **비텍스트 요소에 대한 설명 생성**

**작동 방식:**
- 각 PDF 페이지를 고해상도 이미지(일반적으로 300 DPI)로 렌더링
- 이미지와 함께 상세한 시스템 프롬프트를 VLM에 전달
- 모델이 시각적 콘텐츠를 분석하고 구조화된 Markdown을 출력
- 페이지를 순차적으로 처리하여 하나의 문서로 결합

이 접근 방식은 모델이 사람처럼 문서를 “보고” 이해한다는 점에서 특히 강력하며, 기존 파서가 놓치기 쉬운 맥락, 레이아웃, 시각적 의미를 파악할 수 있습니다.

### 왜 VLM을 선택해야 할까요?

- **최적 용도:** 과학 논문, 인포그래픽, 의료 보고서, 기술 다이어그램
- **장점:** 최고 수준의 정확도, 뛰어난 시각 요소 설명, 공간적 관계 보존, 어떤 레이아웃 복잡성도 처리 가능
- **단점:** 처리 속도가 느림, API 비용 발생(또는 로컬 모델의 경우 상당한 연산 자원 필요)

### 클라우드 vs 로컬 배포

**클라우드 VLM (대부분의 사용자에게 권장):**
- Google Gemini 3 Flash / 2.5 Pro
- OpenAI GPT-4o / GPT-4o-mini
- Anthropic Claude 3.5 Sonnet / Haiku

**로컬 VLM (프라이버시/오프라인 요구 사항용):**
- 성능은 모델 크기와 하드웨어에 크게 의존
- 모델이 클수록 정확도 향상 (예: Qwen2-VL 72B > Qwen2-VL 7B)
- 예시: Qwen2-VL, LLaVA, BakLLaVA, CogVLM
- 상당한 GPU 메모리 필요 (7B 모델은 24GB+, 70B+ 모델은 80GB+)
- Ollama, vLLM, Hugging Face Transformers를 통해 실행 가능

**비용 참고:** Google Gemini는 PDF 변환 작업에 특히 비용 효율적이며, 품질 대비 가격 비율이 우수합니다.

### VLM 변환을 위한 커스텀 시스템 프롬프트

### 한글

In [22]:
SYSTEM_PROMPT = """당신은 PDF 페이지를 마크다운 형식으로 변환하는 데 특화된 문서 파싱 전문가입니다.

**수행 과제:**
제공된 페이지 이미지에서 모든 콘텐츠를 추출하여 깨끗하고 구조화된 마크다운 형식으로 반환하십시오.

**텍스트 추출 규칙:**
1. 오타, 서식, 특수 문자를 포함하여 기록된 텍스트를 있는 그대로 정확하게 유지하십시오.
2. 논리적인 읽기 순서(위에서 아래로, 왼쪽에서 오른쪽으로)를 유지하십시오.
3. 적절한 마크다운 헤더(#, ##, ###)를 사용하여 계층 구조를 보존하십시오.
4. 문단 구분과 줄 간격을 나타난 대로 유지하십시오.
5. 글머리 기호 및 번호 매기기 목록에는 마크다운 리스트(-, *, 1.)를 사용하십시오.
6. 텍스트 강조(강하게(**bold**), 기울임(*italic*), 코드(`code`))를 보존하십시오.
7. 다단(Multi-column) 레이아웃의 경우, 왼쪽 열을 먼저 추출한 다음 오른쪽 열을 추출하십시오.

**표(Tables):**
- 모든 표를 마크다운 표 형식으로 변환하십시오.
- 열 정렬과 구조를 유지하십시오.
- 열 구분에는 |, 헤더 구분에는 -를 사용하십시오.

**수학 공식(Mathematical Formulas):**
- LaTeX 형식으로 변환하십시오: 인라인 `$공식$`, 디스플레이 `$$공식$$`.
- LaTeX 변환이 불확실한 경우, 공식을 명확하게 설명하십시오.

**이미지, 다이어그램, 차트:**
- 마크다운 이미지 자리 표시자를 삽입하십시오: `![설명](image)`
- 다음을 포함하여 상세하고 정보가 풍부한 설명을 제공하십시오:
  * 시각 자료의 유형 (사진, 다이어그램, 차트, 그래프, 삽화 등)
  * 주요 주제 또는 목적
  * 주요 요소, 레이블 또는 데이터 포인트
  * 색상, 패턴 또는 주목할만한 시각적 특징
  * 주변 텍스트와의 맥락 또는 관계
- 차트/그래프의 경우: 축(axes), 데이터 추세 및 주요 수치를 언급하십시오.
- 다이어그램의 경우: 구성 요소와 그 관계를 설명하십시오.

**특수 요소:**
- 각주: 마크다운 각주 구문 `[^1]`을 사용하십시오.
- 인용구: 기록된 대로 보존하십시오.
- 코드 블록: 언어 사양과 함께 세 개의 백틱(```)을 사용하십시오.
- 인용문: 블록 인용에는 `>`를 사용하십시오.
- 링크: 가시적인 경우 `[텍스트](URL)` 형식을 유지하십시오.

**품질 가이드라인:**
- 설명, 주석 또는 메타 정보를 추가하지 마십시오.
- 콘텐츠를 건너뛰거나 요약하지 마십시오.
- 이미지에 없는 텍스트를 지어내거나 환각(Hallucinate)하지 마십시오.
- "여기에 마크다운이 있습니다..."와 같은 서문을 포함하지 마십시오.
- 마크다운 콘텐츠만 출력하고 다른 것은 포함하지 마십시오.

**출력 형식:**
래퍼(wrapper), 코드 블록, 설명이 없는 순수 마크다운을 반환하십시오.
페이지 콘텐츠로 즉시 시작하십시오.
""".strip()

### 영문

In [5]:
# Customize this system prompt based on your PDF type (e.g., academic, technical, legal).
# This template works for 90% of documents—tweak rules as needed for your use case.
SYSTEM_PROMPT = """You are an expert document parser specializing in converting PDF pages to markdown format.

**Your task:**
Extract ALL content from the provided page image and return it as clean, well-structured markdown.

**Text Extraction Rules:**
1. Preserve the EXACT text as written (including typos, formatting, special characters)
2. Maintain the logical reading order (top-to-bottom, left-to-right)
3. Preserve hierarchical structure using appropriate markdown headers (#, ##, ###)
4. Keep paragraph breaks and line spacing as they appear
5. Use markdown lists (-, *, 1.) for bullet points and numbered lists
6. Preserve text emphasis: **bold**, *italic*, `code`
7. For multi-column layouts, extract left column first, then right column

**Tables:**
- Convert all tables to markdown table format
- Preserve column alignment and structure
- Use | for columns and - for headers

**Mathematical Formulas:**
- Convert to LaTeX format: inline `$formula$`, display `$$formula$$`
- If LaTeX conversion is uncertain, describe the formula clearly

**Images, Diagrams, Charts:**
- Insert markdown image placeholder: `![Description](image)`
- Provide a detailed, informative description including:
  * Type of visual (photo, diagram, chart, graph, illustration)
  * Main subject or purpose
  * Key elements, labels, or data points
  * Colors, patterns, or notable visual features
  * Context or relationship to surrounding text
- For charts/graphs: mention axes, data trends, and key values
- For diagrams: describe components and their relationships

**Special Elements:**
- Footnotes: Use markdown footnote syntax `[^1]`
- Citations: Preserve as written
- Code blocks: Use triple backticks with language specification
- Quotes: Use `>` for blockquotes
- Links: Preserve as `[text](url)` if visible

**Quality Guidelines:**
- DO NOT add explanations, comments, or meta-information
- DO NOT skip or summarize content
- DO NOT invent or hallucinate text not present in the image
- DO NOT include "Here is the markdown..." or similar preambles
- Output ONLY the markdown content, nothing else

**Output Format:**
Return raw markdown with no wrapper, no code blocks, no explanations.
Start immediately with the page content.
""".strip()

### Google Gemini를 활용한 구현

이 구현은 페이지별 변환 방식을 보여줍니다.

**참고:** [Gemini API – 이미지 이해](https://ai.google.dev/gemini-api/docs/image-understanding)

In [23]:
import os
import fitz  # PyMuPDF
from google import genai
from google.genai import types

def convert_complex_pdfs_vlm(pdf_path: str, api_key: str, model: str = "gemini-2.5-flash"):
    """
    VLM(Vision-Language Model)을 사용하여 단일 PDF 파일을 마크다운으로 변환합니다.

    PDF의 각 페이지를 이미지로 변환한 뒤, Gemini 모델의 시각 처리 능력을 활용하여
    표, 차트, 이미지 설명이 포함된 구조화된 마크다운을 생성합니다.

    Args:
        pdf_path (str): 대상 PDF 파일 경로.
        api_key (str): Google Gemini API 인증 키.
        model (str): 사용할 Gemini 모델 명칭 (기본값: gemini-2.5-flash).

    Returns:
        dict: 페이지 번호를 키(Key)로, 변환된 마크다운 내용을 값(Value)으로 가지는 딕셔너리.
    """
    # Google GenAI 클라이언트 초기화
    client = genai.Client(api_key=api_key)
    # PyMuPDF를 사용하여 PDF 문서 오픈
    pdf_document = fitz.open(pdf_path)
    markdown_pages = {}

    for page_num in range(pdf_document.page_count):
        try:
            page = pdf_document[page_num]

            # Step 1: 고해상도 분석을 위해 페이지를 300 DPI 이미지로 렌더링
            # 72 DPI 기본값 대비 약 4.16배 확대 (300/72)
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
            img_data = pix.tobytes("png")

            # Step 2: VLM 입력을 위한 이미지 데이터 파트 구성
            image = types.Part.from_bytes(data=img_data, mime_type="image/png")

            # Step 3: Gemini 모델을 호출하여 이미지 기반 콘텐츠 생성
            response = client.models.generate_content(
                config=types.GenerateContentConfig(
                    system_instruction="SYSTEM_PROMPT", # 시스템 프롬프트(별도 정의 필요)
                    temperature=0.1  # 결과의 일관성 및 결정론적 출력을 위해 낮은 온도 설정
                ),
                model=model,
                contents=[
                    "이 PDF 페이지를 깨끗하고 구조화된 마크다운으로 변환하세요. "
                    "모든 텍스트를 추출하고, 이미지를 설명하며, 레이아웃을 최대한 유지하세요.",
                    image
                ],
            )

            # 성공 시 페이지별 텍스트 저장
            markdown_pages[page_num + 1] = response.text
            print(f"✓ 페이지 처리 완료 {page_num + 1}/{pdf_document.page_count}")

        except Exception as e:
            # 개별 페이지 오류 발생 시 에러 메시지를 주석 형태로 포함하여 흐름 유지
            print(f"✗ 페이지 {page_num + 1} 오류: {e}")
            markdown_pages[page_num + 1] = f""

    pdf_document.close()
    return markdown_pages


def batch_convert_complex_pdfs(pdf_folder: str, output_folder: str, api_key: str):
    """
    폴더 내의 모든 PDF 파일을 VLM을 사용하여 일괄 변환합니다.

    Args:
        pdf_folder (str): PDF 파일들이 위치한 폴더 경로.
        output_folder (str): 마크다운 결과물을 저장할 폴더 경로.
        api_key (str): API 키.
    """
    # 출력 폴더가 없으면 생성
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith('.pdf'):
            print(f"\n📄 처리 중: {filename}")
            pdf_path = os.path.join(pdf_folder, filename)
            pdf_name = os.path.splitext(filename)[0]

            # VLM 변환 수행 (페이지별 딕셔너리 반환)
            markdown_pages = convert_complex_pdfs_vlm(pdf_path, api_key)

            # Step 4: 개별 페이지 마크다운을 하나의 파일로 병합
            # 페이지 구분선(---)과 제목(Page N) 추가
            combined_markdown = "\n\n---\n\n".join([
                f"# Page {page_num}\n\n{content}"
                for page_num, content in markdown_pages.items()
            ])

            # 결과 파일 저장
            output_path = os.path.join(output_folder, f"{pdf_name}.md")
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(combined_markdown)

            print(f"✓ 저장 완료: {output_path}")

    print(f"\n🎉 모든 일괄 변환 작업이 완료되었습니다! 위치: '{output_folder}'")

# 예시 실행 코드
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

batch_convert_complex_pdfs("./pdfs", "./md_output_complex", GOOGLE_API_KEY)


📄 처리 중: T_cn202310830097bp.pdf
✓ 페이지 처리 완료 1/13
✓ 페이지 처리 완료 2/13
✓ 페이지 처리 완료 3/13
✓ 페이지 처리 완료 4/13
✓ 페이지 처리 완료 5/13
✗ 페이지 6 오류: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 2.418388283s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'Generate

KeyboardInterrupt: 

## 권장 워크플로

```
┌─────────────────────────────┐
│  PDF 분석                                                │
│  1. 스캔본인가?                                          │
│  2. 이미지가 중요한가?                                   │
│  3. 레이아웃이 복잡한가?                                 │
│  4. 과학적 콘텐츠인가?                                   │      
└──────────┬──────────────────┘
                      │
                      ▼
    ┌──────────────────┐
    │ 디지털 + 단순한가?                 │───예──► PyMuPDF4LLM
    └──────┬───────────┘
                  │아니오
                  ▼
    ┌────────────────────────────────┐
    │ 스캔본 / 표만 포함                                             │
    │ (중요한 시각적 콘텐츠 없음)?                                   │───예──► Docling
    └──────┬─────────────────────────┘
                  │아니오
                  ▼
    ┌──────────────────────────────┐
    │ 복잡한 레이아웃, 차트,                                     │
    │ 다이어그램 또는 해석이 필요한                              │
    │ 시각적 콘텐츠인가?                                         │───예──► VLM (Gemini)
    └──────────────────────────────┘
```

---

## 스캔 문서에 대한 특별 고려사항

**중요:** 스캔된 PDF는 레이아웃 복잡성과 관계없이 항상 OCR이 필요합니다. 스캔된 PDF가 단순한 레이아웃을 가지고 있더라도, 텍스트가 디지털로 선택되지 않기 때문에 반드시 카테고리 2 도구(Docling, Marker, PaddleOCR)로 처리해야 합니다.

**스캔 PDF를 식별하는 방법:**
1. PDF 뷰어에서 텍스트를 선택해 보십시오 — 선택할 수 없다면 스캔본입니다.
2. 파일 크기를 확인하십시오 — 스캔 PDF는 일반적으로 훨씬 큽니다.
3. 이미지 아티팩트나 약간 기울어진 텍스트가 있는지 확인하십시오.

**시나리오별 권장 도구:**
- **영문 스캔 문서:** Docling 또는 Marker
- **다국어 스캔 문서:** PaddleOCR (80개 이상의 언어 지원)
- **저품질 스캔:** 최상의 정확도를 위해 VLM 접근 방식
- **대량 스캔 문서:** Marker (가장 빠른 처리)

---

## 모범 사례

### 1. 항상 샘플 문서로 테스트하십시오
전체 코퍼스를 처리하기 전에 각 카테고리를 대표하는 PDF를 테스트하여 추출 품질을 검증하십시오.

### 2. 품질 검사 구현

In [26]:
from pathlib import Path

def validate_markdown_quality(md_file: Path) -> dict:
    """
    마크다운 파일의 변환 품질을 정량적으로 분석하고 지표를 반환합니다.

    이 함수는 파일 내의 텍스트 밀도, 특수 요소(헤더, 표, 수식)의 존재 여부,
    그리고 전반적인 포맷팅 상태를 검사하여 변환 프로세스의 성공 여부를 평가하는 데 도움을 줍니다.

    Args:
        md_file (Path): 분석할 마크다운 파일의 경로 객체.

    Returns:
        dict: 다음과 같은 품질 지표를 포함하는 딕셔너리:
            - word_count: 전체 단어 수 (공백 기준 분리)
            - has_headers: 제목(Header) 포함 여부
            - has_tables: 마크다운 표(Table) 포함 여부
            - has_formulas: LaTeX 수식(Formula) 포함 여부
            - avg_line_length: 한 줄당 평균 문자 수 (텍스트 밀도)
            - empty_ratio: 전체 줄 대비 빈 줄의 비율 (가독성 및 공백 상태)
    """
    # 마크다운 파일 내용 읽기
    content = md_file.read_text(encoding='utf-8')

    # 공백 문자를 기준으로 분리하여 단어 리스트 생성
    words = content.split()

    # 줄 바꿈 횟수 계산 (분모가 0이 되는 것을 방지하기 위해 최소값 1 설정)
    line_count = max(content.count("\n"), 1)

    return {
        # 1. 정보량 지표: 단어의 총 개수
        "word_count": len(words),

        # 2. 구조화 지표: 제목(#), 표(|), 수식($) 기호 포함 여부 검사
        "has_headers": "#" in content,
        "has_tables": "|" in content,
        "has_formulas": "$" in content,

        # 3. 밀도 지표: 전체 문자 수를 줄 수로 나누어 평균적인 줄의 길이를 계산
        "avg_line_length": len(content) / line_count,

        # 4. 레이아웃 지표: 연속된 줄 바꿈(\n\n)을 통해 문서 내 공백 분포 비율 계산
        "empty_ratio": content.count("\n\n") / line_count
    }

# 실행 예시 (Example usage)
# 'output.md' 파일에 대한 품질 메트릭 측정
quality_metrics = validate_markdown_quality(Path("./md_output_simple/T_cn202310830097bp.md"))
print(f"품질 측정 결과: {quality_metrics}")

quality_metrics = validate_markdown_quality(Path("./md_output_middle/T_cn202310830097bp.md"))
print(f"품질 측정 결과: {quality_metrics}")

품질 측정 결과: {'word_count': 5248, 'has_headers': True, 'has_tables': False, 'has_formulas': False, 'avg_line_length': 23.438947368421054, 'empty_ratio': 0.45052631578947366}
품질 측정 결과: {'word_count': 5293, 'has_headers': True, 'has_tables': False, 'has_formulas': False, 'avg_line_length': 104.13084112149532, 'empty_ratio': 0.4953271028037383}


### 3. 혼합 문서 유형 처리

In [29]:
import fitz  # PyMuPDF

def smart_convert_pdf(pdf_path: str, api_key: str = None):
    """
    PDF의 특성(텍스트 유무, 이미지 밀도 등)을 분석하여 최적의 변환 라이브러리를 동적으로 선택합니다.

    이 함수는 리소스 비용과 변환 정확도 사이의 균형을 맞추기 위한
    '의사 결정 게이트웨이(Decision Gateway)' 역할을 수행합니다.

    Args:
        pdf_path (str): 분석 및 변환할 PDF 파일 경로.
        api_key (str, optional): VLM(Gemini) 사용 시 필요한 API 키.
    """
    # Step 1: 빠른 분석을 위해 PDF 문서 오픈 및 샘플 페이지(첫 페이지) 추출
    doc = fitz.open(pdf_path)
    sample_page = doc[0]

    # Step 2: 텍스트 추출 가능 여부 확인
    # 텍스트 레이어가 없거나 매우 적으면 스캔된 이미지 문서로 판단
    text = sample_page.get_text()
    is_scanned = len(text.strip()) < 50  # 50자 미만일 경우 스캔 문서로 간주

    # Step 3: 이미지 포함 여부 및 개수 확인
    # 문서 내 시각적 요소(차트, 다이어그램 등)의 비중을 파악
    image_count = len(sample_page.get_images())
    has_images = image_count > 2

    # 분석 완료 후 리소스 해제
    doc.close()

    # Step 4: 분석 결과에 따른 라우팅(Routing) 로직

    # CASE 1: 텍스트가 없는 스캔된 문서의 경우
    if is_scanned:
        print("→ [분석 결과] 스캔된 문서 탐지: Docling(OCR) 엔진을 사용합니다.")
        # 이전에 정의된 convert_medium_pdfs_docling 함수 호출
        return convert_medium_pdfs_docling(pdf_path, "md_output")

    # CASE 2: 이미지가 많아 시각적 해석이 중요한 경우
    elif has_images:
        print("→ [분석 결과] 이미지 비중 높음: VLM(Gemini) 엔진을 사용합니다.")
        # 이전에 정의된 convert_complex_pdfs_vlm 함수 호출
        return convert_complex_pdfs_vlm(pdf_path, api_key)

    # CASE 3: 텍스트 레이어가 잘 구성된 일반적인 디지털 PDF의 경우
    else:
        print("→ [분석 결과] 일반 디지털 PDF: PyMuPDF4LLM 엔진을 사용합니다.")
        # 이전에 정의된 convert_simple_pdfs_pymupdf4llm 함수 호출
        return convert_simple_pdfs_pymupdf4llm(pdf_path, "md_output")


smart_convert_pdf('./pdfs/T_cn202310830097bp.pdf')

→ [분석 결과] 일반 디지털 PDF: PyMuPDF4LLM 엔진을 사용합니다.

변환 작업이 완료되었습니다! 결과물 위치: 'output'


---

## 결론

적절한 PDF→Markdown 변환기를 선택하는 것은 RAG 시스템의 성능에 직접적인 영향을 미칩니다.

**핵심 원칙:**
1. 요구 사항을 충족하는 가장 단순한 도구부터 시작하십시오
2. 품질 요구가 있을 때만 고급 방법으로 업그레이드하십시오
3. 혼합 문서 유형에는 하이브리드 접근 방식을 구현하십시오
4. 전체 배포 전에 항상 출력 품질을 검증하십시오
5. 운영 환경에서 비용과 성능을 모니터링하십시오

프로덕션 시스템의 경우, 문서 특성에 따라 최적의 변환 도구를 자동으로 선택하는 지능형 라우팅 시스템 구현을 고려하십시오. 이는 전체 문서 코퍼스 전반에서 일관된 결과를 보장하면서 품질과 비용 효율성을 모두 극대화합니다.

**다음 단계:**
- 특정 문서 유형에 대해 도구를 테스트하십시오
- 품질 대비 속도 대비 비용의 트레이드오프를 측정하십시오
- VLM 접근 방식에 대한 시스템 프롬프트를 반복적으로 개선하십시오
- 품질 보증을 위한 검증 파이프라인을 구축하십시오
- 성능을 모니터링하면서 점진적으로 확장하십시오

## Other PDF Extraction and Processing Tools

- **PDFPlumber** – https://github.com/jsvine/pdfplumber
- **MarkItDown** – https://github.com/microsoft/markitdown
- **Marker** – https://github.com/VikParuchuri/marker
- **Unstructured** – https://github.com/Unstructured-IO/unstructured
- **DeepSeek OCR** – https://github.com/deepseek-ai/DeepSeek-OCR
- **Paddle OCR** – https://github.com/PaddlePaddle/PaddleOCR
- **Surya OCR** – https://github.com/VikParuchuri/surya
- **Dolphin** – https://github.com/bytedance/Dolphin
- **LlamaParse** – https://github.com/run-llama/llama_parse
- **Nougat** – https://github.com/facebookresearch/nougat
- **MinerU** – https://github.com/opendatalab/MinerU